In [1]:
import pandas as pd
import numpy as np

csv_path = 'D:\project\TeamProject1\JeseongMoon\Dataset\mulit_classification_data.csv'
df = pd.read_csv(csv_path)
print(f'{df.shape}')
df.head()

(1941, 34)


,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,42,50,270900,270944,267,17,44,24220,76,108,...,0.8182,-0.2913,0.5822,1,0,0,0,0,0,0
1,645,651,2538079,2538108,108,10,30,11397,84,123,...,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
2,829,835,1553913,1553931,71,8,19,7972,99,125,...,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
3,853,860,369370,369415,176,13,45,18996,99,126,...,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
4,1289,1306,498078,498335,2409,60,260,246930,37,126,...,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
import numpy as np
import tensorflow
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
import sklearn
from sklearn.metrics import accuracy_score, log_loss

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

y = df[['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']]
X = df.iloc[:,:-7]

In [5]:
X.shape

(1941, 27)

In [6]:
X = X.rename(columns={'TypeOfSteel_A300': 'TypeOfSteel'})
X = X.drop(columns=['TypeOfSteel_A400'])
X.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas
0,42,50,270900,270944,267,17,44,24220,76,108,1687,1,80,0.0498,0.2415,0.1818,0.0047,0.4706,1.0000,1.0,2.4265,0.9031,1.6435,0.8182,-0.2913,0.5822
1,645,651,2538079,2538108,108,10,30,11397,84,123,1687,1,80,0.7647,0.3793,0.2069,0.0036,0.6000,0.9667,1.0,2.0334,0.7782,1.4624,0.7931,-0.1756,0.2984
2,829,835,1553913,1553931,71,8,19,7972,99,125,1623,1,100,0.9710,0.3426,0.3333,0.0037,0.7500,0.9474,1.0,1.8513,0.7782,1.2553,0.6667,-0.1228,0.2150
3,853,860,369370,369415,176,13,45,18996,99,126,1353,0,290,0.7287,0.4413,0.1556,0.0052,0.5385,1.0000,1.0,2.2455,0.8451,1.6532,0.8444,-0.1568,0.5212
4,1289,1306,498078,498335,2409,60,260,246930,37,126,1353,0,185,0.0695,0.4486,0.0662,0.0126,0.2833,0.9885,1.0,3.3818,1.2305,2.4099,0.9338,-0.1992,1.0000


In [7]:
X.TypeOfSteel.value_counts()

0    1164
1     777
Name: TypeOfSteel, dtype: int64

In [8]:
pd.set_option('display.expand_frame_repr', False) 
print(X.head())  # 데이터프레임 확인

   X_Minimum  X_Maximum  Y_Minimum  Y_Maximum  Pixels_Areas  X_Perimeter  Y_Perimeter  Sum_of_Luminosity  Minimum_of_Luminosity  Maximum_of_Luminosity  Length_of_Conveyer  TypeOfSteel  Steel_Plate_Thickness  Edges_Index  Empty_Index  Square_Index  Outside_X_Index  Edges_X_Index  Edges_Y_Index  Outside_Global_Index  LogOfAreas  Log_X_Index  Log_Y_Index  Orientation_Index  Luminosity_Index  SigmoidOfAreas
0         42         50     270900     270944           267           17           44              24220                     76                    108                1687            1                     80       0.0498       0.2415        0.1818           0.0047         0.4706         1.0000                   1.0      2.4265       0.9031       1.6435             0.8182           -0.2913          0.5822
1        645        651    2538079    2538108           108           10           30              11397                     84                    123                1687            1   

In [9]:
y["Type"] = y[['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']].idxmax(axis=1)
y = y.drop(columns=['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults'])
y.head()

c:\Users\82105\anaconda3\envs\project1\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Type
0,Pastry
1,Pastry
2,Pastry
3,Pastry
4,Pastry


In [10]:
y.Type.value_counts()

Other_Faults    673
Bumps           402
K_Scatch        391
Z_Scratch       190
Pastry          158
Stains           72
Dirtiness        55
Name: Type, dtype: int64

In [11]:
print(X.shape, y.shape)

(1941, 26) (1941, 1)


In [12]:
y_dataframe = df.iloc[:,-7:]

In [13]:
y_list = []
for i in range(y_dataframe.shape[0]):
    if y_dataframe["Pastry"].values[i] == 1:
        y_list.append("Pastry")
    elif y_dataframe["Z_Scratch"].values[i] == 1:
        y_list.append("Z_Scratch")
    elif y_dataframe["K_Scatch"].values[i] == 1:
        y_list.append("K_Scatch")
    elif y_dataframe["Stains"].values[i] == 1:
        y_list.append("Stains")
    elif y_dataframe["Dirtiness"].values[i] == 1:
        y_list.append("Dirtiness")
    elif y_dataframe["Bumps"].values[i] == 1:
        y_list.append("Bumps")
    else:
        y_list.append("Other_Faults")

y_list = np.array(y_list)
y_list.shape

(1941,)

In [14]:
label = y['Type']
print(label.unique())

['Pastry' 'Z_Scratch' 'K_Scatch' 'Stains' 'Dirtiness' 'Bumps'
 'Other_Faults']


In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(label)
label_encoded = le.transform(label)
le_y = pd.DataFrame(label_encoded, columns=['label'])
y = le_y
print(y.value_counts())

label
3        673
0        402
2        391
6        190
4        158
5         72
1         55
dtype: int64


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 83)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape, )

(1552, 26) (389, 26) (1552, 1) (389, 1)


In [17]:
from sklearn import metrics
import numpy as np

def get_metrics(model, X, y):
    y_pred=model.predict(X)
    
    Precision    = metrics.precision_score (y, y_pred, average="weighted")
    Recall       = metrics.recall_score    (y, y_pred, average="weighted")
    Accuracy     = metrics.accuracy_score  (y, y_pred)
    F1           = metrics.f1_score        (y, y_pred, average="weighted")
    
    print(f'Precision Score  : {Precision}')
    print(f'Recall Score     : {Recall}')
    print(f'Accuracy Score   : {Accuracy}')
    print(f'F1 Score         : {F1}')
    return Precision, Recall, Accuracy, F1

In [18]:
from sklearn import metrics
import numpy as np

def get_metrics(model, X, y):
    y_pred=model.predict(X)
    
    Precision    = metrics.precision_score (y, y_pred, average="weighted")
    Recall       = metrics.recall_score    (y, y_pred, average="weighted")
    Accuracy     = metrics.accuracy_score  (y, y_pred)
    F1           = metrics.f1_score        (y, y_pred, average="weighted")
    
    print(f'Precision Score  : {Precision}')
    print(f'Recall Score     : {Recall}')
    print(f'Accuracy Score   : {Accuracy}')
    print(f'F1 Score         : {F1}')
    return Precision, Recall, Accuracy, F1

In [19]:
from sklearn.ensemble import RandomForestClassifier
model_rf_all = RandomForestClassifier(random_state=83)
model_rf_all.fit(X_train, y_train)

rf_precision_train, rf_recall_train, rf_accuracy_train, rf_f1_train = get_metrics(model_rf_all, X_train, y_train)
print('='*100)
rf_precision_test, rf_recall_test, rf_accuracy_test, rf_f1_test = get_metrics(model_rf_all, X_test, y_test)

c:\Users\82105\anaconda3\envs\project1\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


Precision Score  : 1.0
Recall Score     : 1.0
Accuracy Score   : 1.0
F1 Score         : 1.0
Precision Score  : 0.7992110315751305
Recall Score     : 0.7866323907455013
Accuracy Score   : 0.7866323907455013
F1 Score         : 0.7892523019803969


In [20]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from sklearn.metrics import accuracy_score, log_loss

In [24]:
from sklearn.utils import class_weight

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='LeakyReLU', input_shape=(X_train.shape[1],)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.001),  # 드롭아웃 추가
    tf.keras.layers.Dense(256, activation='LeakyReLU', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001)),  # L1, L2 정규화 적용
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.001),  # 드롭아웃 추가
    tf.keras.layers.Dense(128, activation='LeakyReLU', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001)),  # L1, L2 정규화 적용
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.001),  # 드롭아웃 추가
    tf.keras.layers.Dense(64, activation='LeakyReLU', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001)),  # L1, L2 정규화 적용
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.001),  # 드롭아웃 추가
    tf.keras.layers.Dense(32, activation='LeakyReLU', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001)),  # L1, L2 정규화 적용
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.001),  # 드롭아웃 추가
    tf.keras.layers.Dense(16, activation='LeakyReLU', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001)),  # L1, L2 정규화 적용
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.001),  # 드롭아웃 추가
    tf.keras.layers.Dense(7, activation='softmax')
])


optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 학습 자동 중단 설정
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=40, mode='min')

# 모델 학습
model.fit(X_train, tf.keras.utils.to_categorical(y_train), epochs=100, batch_size=5, verbose=1, 
          validation_data=(X_test, tf.keras.utils.to_categorical(y_test)), callbacks=[rlrp])

# 모델 평가
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # 클래스 예측값으로 변환
accuracy = accuracy_score(y_test, y_pred_classes)
loss = log_loss(y_test, y_pred)
print(f'Accuracy: {accuracy}\nLog Loss: {loss}')

Epoch 1/100
311/311 [==============================] - 3s 5ms/step - loss: 9.9968 - accuracy: 0.3628 - val_loss: 8.3368 - val_accuracy: 0.4679
Epoch 2/100
311/311 [==============================] - 1s 4ms/step - loss: 7.1005 - accuracy: 0.4304 - val_loss: 5.9750 - val_accuracy: 0.4910
Epoch 3/100
311/311 [==============================] - 1s 3ms/step - loss: 5.3293 - accuracy: 0.4356 - val_loss: 4.7623 - val_accuracy: 0.4550
Epoch 4/100
311/311 [==============================] - 1s 4ms/step - loss: 4.2688 - accuracy: 0.4323 - val_loss: 3.9909 - val_accuracy: 0.4961
Epoch 5/100
311/311 [==============================] - 1s 3ms/step - loss: 3.7544 - accuracy: 0.4452 - val_loss: 3.7265 - val_accuracy: 0.4473
Epoch 6/100
311/311 [==============================] - 1s 4ms/step - loss: 3.5190 - accuracy: 0.4452 - val_loss: 3.3391 - val_accuracy: 0.4370
Epoch 7/100
311/311 [==============================] - 1s 3ms/step - loss: 3.0840 - accuracy: 0.4394 - val_loss: 2.9203 - val_accuracy: 0.4679

In [25]:
from sklearn.utils import class_weight

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='ReLU', input_shape=(X_train.shape[1],)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation='ReLU'),  
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation='ReLU'),  
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation='ReLU'),  
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation='ReLU'),  
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(16, activation='ReLU'),  
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(7, activation='softmax')
])


optimizer = tf.keras.optimizers.Adamax()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 학습 자동 중단 설정
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=40, mode='min')

# 모델 학습
model.fit(X_train, tf.keras.utils.to_categorical(y_train), epochs=100, batch_size=5, verbose=1, 
          validation_data=(X_test, tf.keras.utils.to_categorical(y_test)), callbacks=[rlrp])

# 모델 평가
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # 클래스 예측값으로 변환
accuracy = accuracy_score(y_test, y_pred_classes)
loss = log_loss(y_test, y_pred)
print(f'Accuracy: {accuracy}\nLog Loss: {loss}')

Epoch 1/100
311/311 [==============================] - 2s 3ms/step - loss: 1.9756 - accuracy: 0.2674 - val_loss: 1.9385 - val_accuracy: 0.2751
Epoch 2/100
311/311 [==============================] - 1s 3ms/step - loss: 1.7251 - accuracy: 0.3537 - val_loss: 1.7632 - val_accuracy: 0.3342
Epoch 3/100
311/311 [==============================] - 1s 3ms/step - loss: 1.6305 - accuracy: 0.4014 - val_loss: 1.7075 - val_accuracy: 0.4010
Epoch 4/100
311/311 [==============================] - 1s 3ms/step - loss: 1.5735 - accuracy: 0.4162 - val_loss: 1.6552 - val_accuracy: 0.4370
Epoch 5/100
311/311 [==============================] - 1s 4ms/step - loss: 1.5369 - accuracy: 0.4356 - val_loss: 1.5140 - val_accuracy: 0.4730
Epoch 6/100
311/311 [==============================] - 1s 3ms/step - loss: 1.5464 - accuracy: 0.4265 - val_loss: 1.5417 - val_accuracy: 0.4679
Epoch 7/100
311/311 [==============================] - 1s 2ms/step - loss: 1.5114 - accuracy: 0.4388 - val_loss: 1.4785 - val_accuracy: 0.4781

In [26]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='ReLU', input_shape=(X_train.shape[1],)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation='ReLU'),  
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation='ReLU'),  
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation='ReLU'),  
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation='ReLU'),  
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(16, activation='ReLU'),  
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(7, activation='softmax')
])


optimizer = tf.keras.optimizers.SGD()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 학습 자동 중단 설정
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=40, mode='min')

# 모델 학습
model.fit(X_train, tf.keras.utils.to_categorical(y_train), epochs=100, batch_size=5, verbose=1, 
          validation_data=(X_test, tf.keras.utils.to_categorical(y_test)), callbacks=[rlrp])

# 모델 평가
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # 클래스 예측값으로 변환
accuracy = accuracy_score(y_test, y_pred_classes)
loss = log_loss(y_test, y_pred)
print(f'Accuracy: {accuracy}\nLog Loss: {loss}')

Epoch 1/100
311/311 [==============================] - 2s 3ms/step - loss: 1.7620 - accuracy: 0.3351 - val_loss: 1.6034 - val_accuracy: 0.4242
Epoch 2/100
311/311 [==============================] - 1s 3ms/step - loss: 1.5468 - accuracy: 0.4555 - val_loss: 1.4867 - val_accuracy: 0.4704
Epoch 3/100
311/311 [==============================] - 1s 3ms/step - loss: 1.5303 - accuracy: 0.4388 - val_loss: 1.4969 - val_accuracy: 0.4679
Epoch 4/100
311/311 [==============================] - 1s 3ms/step - loss: 1.5274 - accuracy: 0.4439 - val_loss: 1.5056 - val_accuracy: 0.4833
Epoch 5/100
311/311 [==============================] - 1s 3ms/step - loss: 1.4823 - accuracy: 0.4562 - val_loss: 1.5088 - val_accuracy: 0.4396
Epoch 6/100
311/311 [==============================] - 1s 3ms/step - loss: 1.4792 - accuracy: 0.4394 - val_loss: 1.4603 - val_accuracy: 0.4781
Epoch 7/100
311/311 [==============================] - 1s 2ms/step - loss: 1.4897 - accuracy: 0.4594 - val_loss: 1.5093 - val_accuracy: 0.4679

In [29]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='ReLU', input_shape=(X_train.shape[1],)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation='ReLU'),  
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(7, activation='softmax')
])


optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 학습 자동 중단 설정
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=40, mode='min')

# 모델 학습
model.fit(X_train, tf.keras.utils.to_categorical(y_train), epochs=100, batch_size=2, verbose=1, 
          validation_data=(X_test, tf.keras.utils.to_categorical(y_test)), callbacks=[rlrp])

# 모델 평가
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # 클래스 예측값으로 변환
accuracy = accuracy_score(y_test, y_pred_classes)
loss = log_loss(y_test, y_pred)
print(f'Accuracy: {accuracy}\nLog Loss: {loss}')

Epoch 1/100
776/776 [==============================] - 2s 2ms/step - loss: 1.7387 - accuracy: 0.3595 - val_loss: 2.2191 - val_accuracy: 0.3470
Epoch 2/100
776/776 [==============================] - 1s 1ms/step - loss: 1.5990 - accuracy: 0.4343 - val_loss: 2.0305 - val_accuracy: 0.3933
Epoch 3/100
776/776 [==============================] - 1s 1ms/step - loss: 1.5679 - accuracy: 0.4336 - val_loss: 2.3401 - val_accuracy: 0.3599
Epoch 4/100
776/776 [==============================] - 1s 1ms/step - loss: 1.5424 - accuracy: 0.4265 - val_loss: 2.4081 - val_accuracy: 0.3805
Epoch 5/100
776/776 [==============================] - 1s 1ms/step - loss: 1.5362 - accuracy: 0.4381 - val_loss: 2.6595 - val_accuracy: 0.3882
Epoch 6/100
776/776 [==============================] - ETA: 0s - loss: 1.5309 - accuracy: 0.44 - 1s 1ms/step - loss: 1.5301 - accuracy: 0.4465 - val_loss: 2.6912 - val_accuracy: 0.3933
Epoch 7/100
776/776 [==============================] - 1s 1ms/step - loss: 1.5713 - accuracy: 0.4195

In [30]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='ReLU', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(256, activation='ReLU'),  
    tf.keras.layers.Dense(128, activation='ReLU'),  
    tf.keras.layers.Dense(64, activation='ReLU'),  
    tf.keras.layers.Dense(32, activation='ReLU'),  
    tf.keras.layers.Dense(16, activation='ReLU'),  
    tf.keras.layers.Dense(7, activation='softmax')
])


optimizer = tf.keras.optimizers.Adamax()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 학습 자동 중단 설정
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=40, mode='min')

# 모델 학습
model.fit(X_train, tf.keras.utils.to_categorical(y_train), epochs=100, batch_size=5, verbose=1, 
          validation_data=(X_test, tf.keras.utils.to_categorical(y_test)), callbacks=[rlrp])

# 모델 평가
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # 클래스 예측값으로 변환
accuracy = accuracy_score(y_test, y_pred_classes)
loss = log_loss(y_test, y_pred)
print(f'Accuracy: {accuracy}\nLog Loss: {loss}')

Epoch 1/100
311/311 [==============================] - 2s 3ms/step - loss: 2327.1394 - accuracy: 0.2726 - val_loss: 2.1857 - val_accuracy: 0.3805
Epoch 2/100
311/311 [==============================] - 1s 3ms/step - loss: 5.0689 - accuracy: 0.3363 - val_loss: 4.0260 - val_accuracy: 0.3650
Epoch 3/100
311/311 [==============================] - 1s 3ms/step - loss: 7.7031 - accuracy: 0.3415 - val_loss: 1.8649 - val_accuracy: 0.3650
Epoch 4/100
311/311 [==============================] - 1s 3ms/step - loss: 1.8531 - accuracy: 0.3421 - val_loss: 1.8294 - val_accuracy: 0.3650
Epoch 5/100
311/311 [==============================] - 1s 2ms/step - loss: 1.8209 - accuracy: 0.3421 - val_loss: 1.7988 - val_accuracy: 0.3650
Epoch 6/100
311/311 [==============================] - 1s 2ms/step - loss: 1.7953 - accuracy: 0.3421 - val_loss: 1.7757 - val_accuracy: 0.3650
Epoch 7/100
311/311 [==============================] - 1s 3ms/step - loss: 1.7753 - accuracy: 0.3421 - val_loss: 1.7565 - val_accuracy: 0.3